In [2]:
import cvxpy as cp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
import numpy as np
m = 30
n = 20
nb = 6
nh = 7
ns = 7
L = 2
sigma = .1
np.random.seed(1)
A = 0.06*np.random.normal(0, 1, size=(m, n))
Sigma = A.T@A

In [6]:
def robust_approach():
  w = cp.Variable(n)

  end_buy  =  nb
  end_hold =  nb + nh
  end_sell =  nb + nh + ns

  wb = w[0:end_buy]
  wh = w[end_buy:end_hold]
  ws = w[end_hold:end_sell]

  obj = cp.sum(wb) -cp.sum(ws) - cp.norm1(wh)

  constraints = [
      cp.sum(w) == 1,
      cp.norm1(w) <= L,
      cp.quad_form(w, Sigma) <= (sigma**2),
      ws <= 0,
      wb >= 0
  ]

  problem = cp.Problem(cp.Maximize(obj), constraints)
  cost = problem.solve()

  return cost, w.value

cost, w_star_robust = robust_approach()
print(f"Robust return:{cost}")
print(f"Robust w_star:{np.round(w_star_robust, 3)}")

Robust return:1.2198227254209897
Robust w_star:[ 0.247  0.247  0.131  0.231  0.164  0.112  0.     0.025  0.115 -0.021
  0.     0.    -0.    -0.17  -0.    -0.    -0.024 -0.    -0.    -0.056]


### c

In [7]:
def naive_approach():
  w = cp.Variable(n)

  end_buy  =  nb
  end_hold =  nb + nh
  end_sell =  nb + nh + ns

  mu = np.zeros(n)
  mu[0:end_buy] = 1
  mu[end_hold:end_sell] = -1

  objective = mu @ w

  constraints = [
      cp.sum(w) == 1,
      cp.norm1(w) <= L,
      cp.quad_form(w, Sigma) <= (sigma**2),
  ]

  problem = cp.Problem(cp.Maximize(objective), constraints)
  cost = problem.solve()
  return cost, w.value

cost, w_star_naive = naive_approach()
print(f"Naive return:{cost}")
print(f"Naive w_star:{np.round(w_star_naive, 3)}")

Naive return:1.5544257857755333
Naive w_star:[ 0.258  0.284  0.127  0.273  0.157  0.133  0.039  0.034  0.102 -0.09
  0.024  0.    -0.019 -0.213 -0.     0.    -0.056  0.069  0.    -0.122]
